# iTEBD algorithm for the Ising model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy.linalg import svd, expm

Definitions of the Pauli matrices

In [ ]:
sx = np.array([[0., 1.], [1., 0.]])
sy = np.array([[0.,-1.j], [1.j,0.]])
sz = np.array([[1., 0.], [0., -1.]])
s0 = np.eye(2)

Definition of the Hamilton operator and the imaginary time evolution operator that we need in the iTEBD step

In [ ]:
def init_ising(J, hx, hz, L, delta):
    """ Returns the Hamilton opertator and the Theta"""
    d = 2
    U_bond = []
    H_bond = []
    for i in range(L):
        H = -J * np.kron(sz, sz) - hx * (np.kron(sx, s0) + np.kron(s0, sx)) / 2. - hz * (
            np.kron(sz, s0) + np.kron(s0, sz)) / 2.
        H_bond.append(np.reshape(H, (d, d, d, d)))
        U_bond.append(np.reshape(expm(-delta * H), (d, d, d, d)))
    return U_bond, H_bond

Calculation of expectation values for operators that act on bonds (on two sites) or on a single site

In [ ]:
def bond_expectation(gamma, lambd, O_list):
    """ Expectation value for a bond operator """
    E = []
    L = len(gamma)
    for i_bond in range(L):
        BB = np.tensordot(gamma[i_bond], gamma[np.mod(i_bond + 1, L)], axes=(2, 1))
        sBB = np.tensordot(np.diag(lambd[np.mod(i_bond, L)]), BB, axes=(1, 1))
        C = np.tensordot(sBB, O_list[i_bond], axes=([1, 2], [2, 3]))
        sBB = np.conj(sBB)
        E.append(np.squeeze(np.tensordot(sBB, C, axes=([0, 3, 1, 2], [0, 1, 2, 3]))).item())
    return E

def site_expectation(gamma, lambd, O_list):
    """ Expectation value for a site operator """
    E = []
    L = len(gamma)
    for isite in range(0, L):
        sB = np.tensordot(np.diag(lambd[np.mod(isite, L)]), gamma[isite], axes=(1, 1))
        C = np.tensordot(sB, O_list[isite], axes=(1, 0))
        sB = sB.conj()
        E.append(np.squeeze(np.tensordot(sB, C, axes=([0, 1, 2], [0, 2, 1]))).item())
    return (E)

Function that performs the sweep for a two-site system
This is the actual algorithm that we looked at in the lecture.

**TODO**:
- Have a look at the function sweep implemented below and try to understand why the tensors are contracted in this way

In [ ]:
def sweep(gamma, lambd, U_bond, chi):
    """ Perform one time evolution step """
    L = len(gamma)
    d = gamma[0].shape[0]
    for k in [0,1]:
        ia = k
        ib = np.mod(k + 1, L)
        chia = gamma[ia].shape[1]
        chic = gamma[ib].shape[2]

        # Construct theta matrix and time evolution #
        theta = np.tensordot(gamma[ia], gamma[ib], axes=(2, 1))  # i a j c
        theta = np.tensordot(U_bond[k], theta, axes=([2, 3], [0, 2]))  # i' j' a c
        theta = np.tensordot(np.diag(lambd[ia]), theta, axes=([1, 2]))  # a i' j' c
        theta = np.reshape(np.transpose(theta, (1, 0, 2, 3)),
                           (d * chia, d * chic))  # (i' a) (j' c)

        # Singular value decomposition #
        U, S, Vd = svd(theta, full_matrices=0, lapack_driver='gesvd')
        # (S is sorted descending)
        chi2 = min(np.sum(S > 10.**(-10)), chi)

        S = S[:chi2]
        invsq = np.sqrt(sum(S**2))
        U = U[:, :chi2]  # (i' a) b
        Vd = Vd[:chi2, :]  # b (j' c)

        # Obtain the new values for gamma and lambda #
        lambd[ib] = S / invsq
        

        U = np.reshape(U, (d, chia, chi2))
        U = np.transpose(np.tensordot(np.diag(lambd[ia]**(-1)), U, axes=(1, 1)), (1, 0, 2))
        gamma[ia] = np.tensordot(U, np.diag(lambd[ib]), axes=(2, 0))
        gamma[ib] = np.transpose(np.reshape(Vd, (chi2, d, chic)), (1, 0, 2))

Convergence of energy for a given value of $\delta$
We want to see that the energy actually converges against the correct value.
The function "f" defined below calculates the exact solution for the energy for the given parameters.

**TODO**:
- Measure the energy in every step of the loop and store the values
- Plot the values of energy (and the correct value)

In [ ]:
%matplotlib notebook

chi=10
J=1. #Do not change this value since the exact solution uses this as energy scale
hx=0.5
hz=0.0 #Do not change this value since we cannot solve the model exactly otherwise
n_imaginary=200
delta=0.001

# Generate a random initial state
gamma=[np.random.rand(2,chi,chi)]*2
lambd=[np.random.rand(chi)]*2

N = int(n_imaginary / np.sqrt(delta))
U_bond, H_bond = init_ising(J, hx, hz, 2, delta)
for i in range(N):
    sweep(gamma, lambd, U_bond, chi)
    #Measure here
    
# Calculate exact groundstate energy
def f(k, hx):
    return -np.sqrt(1 + hx**2 - 2 * hx * np.cos(k)) / np.pi
E0_exact = integrate.quad(f, 0, np.pi, args=(hx, ))[0]

fig,ax0 = plt.subplots()
#Plot the energy
ax0.set_xlabel('iteration')
ax0.set_ylabel('E')

Convergence of the energy for varying values of $\delta$
In the next cell, we calculate the energy for different values of the timestep $\delta$.

**TODO**
- Evaluate the error of the energy after the last iteration of the iTEBD algorithm and store the values
- Plot the dependence of the error in energy on $\delta$

In [ ]:
chi=10
J=1.
hx=0.5
hz=0.0
n_imaginary=100

%matplotlib notebook
"""run imaginary time evolution for the infinite ising chain with iTEBD"""
# Generate a random initial state
gamma=[np.random.rand(2,chi,chi)]*2
lambd=[np.random.rand(chi)]*2

#Use different values of delta to show the influence of different timesteps
deltavec=[0.1, 0.01, 0.001, 0.0001]

for delta in deltavec:
    N = int(n_imaginary / np.sqrt(delta))
    U_bond, H_bond = init_ising(J, hx, hz, 2, delta)
    for i in range(N):
        sweep(gamma, lambd, U_bond, chi)
    m = np.mean(site_expectation(gamma, lambd, 2 * [sz]))
    #Calculate the energy (again)
    
    # Calculate exact groundstate energy
    if hz == 0 and np.abs(J) == 1:
        def f(k, hx):
            return -np.sqrt(1 + hx**2 - 2 * hx * np.cos(k)) / np.pi
        E0_exact = integrate.quad(f, 0, np.pi, args=(hx, ))[0]
        ### Uncomment if the energy calculation is inserted ###
        #print("--> delta = {D:.6f}, E = {E:2.6f} (dE = {dE:2.2e}), m = {m:2.6f}".format(D=delta, E=E,dE=E - E0_exact,m=np.abs(m)))
    #else:
        #print("--> delta = {D:.6f}, m = {m:2.6f}, E = {E:2.6f}".format(D=delta, m=np.abs(m), E=E))
        
if hz==0 and np.abs(J) == 1:
    fig, axes=plt.subplots()
    #Plot the error in energy and delta
    #NOTE: A loglog plot is more suitable than a linear plot
    axes.set_xlabel(r'$\delta$')
    axes.set_ylabel(r'dE')